# dev_utils

> TODO fill in description

In [ ]:
#| default_exp dev_utils

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from pathlib import Path
import pickle
import functools
import inspect
from typing import Callable, Type
import sys

import fbdev
from fbdev._utils import get_function_from_py_file
from fbdev.comp import BaseComponent
from fbdev.complib import ExecComponent
from fbdev.runtime import BatchExecutor

In [ ]:
#|hide
show_doc(fbdev.dev_utils.is_in_repl)

---

### is_in_repl

>      is_in_repl ()

In [ ]:
#|export
def is_in_repl():
    return hasattr(sys, 'ps1')

In [ ]:
assert is_in_repl()

In [ ]:
#|hide
show_doc(fbdev.dev_utils.method_from_py_file)

---

### method_from_py_file

>      method_from_py_file (file_path:str)

In [ ]:
#|export
def method_from_py_file(file_path:str):
    def decorator(orig_func):
        args = list(inspect.signature(orig_func).parameters.keys())
        is_async = inspect.iscoroutinefunction(orig_func)
        new_func = get_function_from_py_file(file_path, func_name=orig_func.__name__, args=args, is_async=is_async)
        if is_async:
            @functools.wraps(orig_func)
            async def wrapped_method(*args, **kwargs):
                await new_func(*args, **kwargs)
                await orig_func(*args, **kwargs)
        else:
            @functools.wraps(orig_func)
            def wrapped_method(*args, **kwargs):
                new_func(*args, **kwargs)
                orig_func(*args, **kwargs)
        return wrapped_method
    return decorator

In [ ]:
#|hide
show_doc(fbdev.dev_utils.get_nb_testing_comp_process)

---

### get_nb_testing_comp_process

>      get_nb_testing_comp_process
>                                   (component_type:Type[fbdev.comp.base_compone
>                                   nt.BaseComponent], configs={}, **inputs)

In [ ]:
#|export
def get_nb_testing_comp_process(component_type: Type[BaseComponent], configs={}, **inputs) -> BaseComponent:
    comp_process = component_type()
    
    async def packet_putter(port_id, val):
        while True:
            await comp_process.ports[port_id]._put_value(val)
        
    async def packet_getter(port_id):
        while True:
            await comp_process.ports[port_id]._get()
        
    coros = [packet_putter(comp_process.ports.input[port_name].id, val) for port_name,val in inputs.items()]
    coros += [packet_putter(comp_process.ports.config[port_name].id, val) for port_name,val in configs.items()]
    coros += [packet_getter(comp_process.ports.output[port_name].id) for port_name in comp_process.ports.output.keys()]
    tasks = [asyncio.create_task(coro) for coro in coros]
    
    async def exception_catcher(tasks):
        while tasks:
            done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_EXCEPTION)
            for task in done:
                if task.exception():
                    print(f"An exception occurred: {task.exception()}")
            tasks = list(pending)
            
    asyncio.create_task(exception_catcher(tasks))
    
    return comp_process